In [38]:
import math

import pandas as pd
from neupy import algorithms
import numpy as np
import json
from tqdm import tqdm
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from numpy import sqrt
import math

In [39]:
f = open("backtest.json")
result = json.load(f)
result = result['strategy']['BTCStrategy003']['trades']
first_timestamp = result[0]['open_timestamp']
last_timestamp = result[-1]['open_timestamp']

df = []
orders_df = []
for trade in result:
    df.append([trade['close_date'], trade['profit_ratio']])
    for order in trade['orders']:
        orders_df.append([order['order_filled_timestamp'], order['safe_price'], order['ft_order_side']])
df = pd.DataFrame(df, columns=['date', 'ret'])
orders_df = pd.DataFrame(orders_df, columns=['timestamp', 'price', 'type'])
buys_df = orders_df[orders_df['type'] == 'buy']
sells_df = orders_df[orders_df['type'] == 'sell']
# make a df with buys and sells

In [40]:
f = open("BTC_USDT-3m.json")
btc_df = pd.DataFrame.from_records(json.load(f), columns=['timestamp', 'open', 'high', 'low', 'close', 'vol'])
btc_df = btc_df[btc_df['timestamp'] >= first_timestamp]
btc_df = btc_df[btc_df['timestamp'] <= last_timestamp]

In [41]:
fig = make_subplots()
fig.add_trace(go.Scatter(x=df['date'], y=df['ret'].cumsum(), mode='lines'))
fig.update_layout(title="Cumsum rets in time. Amount of trades=" + str(len(df)), yaxis_tickformat=".00%")
fig.show()

fig = make_subplots()

fig.add_trace(go.Scatter(x=btc_df['timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000)), y=btc_df['close'],
                         name="Close price in time", mode='lines'))
fig.add_trace(go.Scatter(x=buys_df['timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000)),
                         y=buys_df['price'], name="Buys", mode="markers"))
fig.add_trace(go.Scatter(x=sells_df['timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000)),
                         y=sells_df['price'], name="Sells", mode="markers"))

fig.update_layout(
    title="Amount of trades=" + str(len(df)))
fig.show()